Credit: Codes were adopted from https://github.com/Sungwon-Han/urban_score

Sungwon Han, Donghyun Ahn, Sungwon Park, Jeasurk Yang, Susang Lee, Jihee Kim, Hyunjoo Yang, Sangyoon Park, and Meeyoung Cha. 2020. Learning to Score Economic Development from Satellite Imagery. In Proceedings of the 26th ACM SIGKDD International Conference on Knowledge Discovery &amp; Data Mining (KDD '20). Association for Computing Machinery, New York, NY, USA, 2970–2979. https://doi.org/10.1145/3394486.3403347

In [ ]:
!pip install finch-clust

In [ ]:
import sys
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from finch import FINCH

for target_index in ['index002','index004','index006','index008','index01']:
    # Define the path to your data file
    data_path_eviction = './AirbnbData/Chicago/train-gridsize-{}.csv'.format(str(target_index)[5:])

    # Load the data using pandas
    data_eviction_df = pd.read_csv(data_path_eviction)  # Adjust if your file has a different delimiter or encoding
    data_eviction_df = data_eviction_df.drop(columns=[target_index, 'label'])
    # Convert DataFrame to numpy array
    data_eviction = data_eviction_df.values

    # Handle missing values
    imputer = SimpleImputer(strategy='mean')
    data_eviction = imputer.fit_transform(data_eviction)

    # Scale the data
    scaler = preprocessing.MinMaxScaler()
    data_eviction = scaler.fit_transform(data_eviction)

    # Run FINCH clustering algorithm
    c_eviction, num_clust_eviction, req_c_eviction = FINCH(data_eviction, distance='euclidean')

    # Prepare data for CSV
    # Create a DataFrame with original data, cluster labels, and partition information
    cluster_df = pd.DataFrame(data_eviction_df)

    # Add partition information
    for i in range(c_eviction.shape[1]):
        cluster_df[f'Partition_{i+1}'] = c_eviction[:, i]

    # Define the path for the output CSV file
    output_csv_path = './AirbnbData/Chicago/grid{}results.csv'.format(str(target_index)[5:])  # Get the output file path from command line argument

    # Save the DataFrame to a CSV file
    cluster_df.to_csv(output_csv_path, index=False)



for target_index in ['index002','index004','index006','index008','index01']:
            print(target_index)
            train_data = pd.read_csv('./AirbnbData/Chicago/train-gridsize-{}.csv'.format(str(target_index)[5:]))
            cluster_labels = list(pd.read_csv('./AirbnbData/Chicago/grid{}results.csv'.format(str(target_index)[5:]))['Partition_1'].values)
            result = pd.DataFrame({"y_train":list(train_data['label'].values), "cluster_labels":cluster_labels})
            result.to_csv('./AirbnbData/Chicago/finch-train-gridsize-{}-c.csv'.format(str(target_index)[5:]))

In [ ]:
import numpy as np
import pandas as pd
import itertools

for target_index in ['index002','index004','index006','index008','index01']:
    census_geoid_df = pd.read_csv('./AirbnbData/Chicago/grid_population_mapping.csv')
    census_geoid_df['GEOID'] = census_geoid_df['GEOID'].map(lambda x: str(x))
    acs_df = pd.read_csv('./AirbnbData/Chicago/S2503Data.csv', skiprows = [1])[['S2503_C01_028E','S2503_C01_032E','S2503_C01_036E','S2503_C01_040E','S2503_C01_044E','GEO_ID']]
    acs_df['CensusPop'] = acs_df['S2503_C01_028E'] + acs_df['S2503_C01_032E'] + acs_df['S2503_C01_036E'] + acs_df['S2503_C01_040E']+ acs_df['S2503_C01_044E']
    acs_df['GEOID'] = acs_df['GEO_ID'].map(lambda x: str(x).replace("1400000US",""))
    acs_df = acs_df[['GEOID','CensusPop']]
    census_geoid_df = pd.merge(census_geoid_df, acs_df, on="GEOID", how = "left")

    train_data_class = pd.read_csv('./AirbnbData/Chicago/finch-train-gridsize-{}-c.csv'.format(str(target_index)[5:]))[['cluster_labels']]
    cnum = len(list(set(list(train_data_class['cluster_labels'].values))))
    print(target_index, cnum)
    train_data = pd.read_csv('./AirbnbData/Chicago/train-gridsize-{}.csv'.format(str(target_index)[5:]))
    index_cluster_label_df = pd.concat([train_data, train_data_class], axis=1)

    census_geoid_df = pd.merge(census_geoid_df, index_cluster_label_df, on=target_index, how = "left")
    census_geoid_df = census_geoid_df.loc[census_geoid_df['cluster_labels'].notnull()]
    final_data = census_geoid_df.groupby(['GEOID','CensusPop','cluster_labels']).agg(num_grids = (target_index,"nunique")).reset_index()
    lst, lst2 = list(set(census_geoid_df['GEOID'].values)), list(range(cnum))
    final_data = pd.merge(final_data, pd.DataFrame(list(itertools.product(lst, lst2)), columns=['GEOID','cluster_labels']), on=['GEOID', 'cluster_labels'], how="right")
    final_data.sort_values(by=['GEOID', 'cluster_labels'], inplace = True)
    final_data = final_data.pivot(index="GEOID", columns="cluster_labels", values="num_grids").reset_index()
    final_data = pd.merge(final_data, acs_df, on="GEOID", how="left")
    final_data = final_data.fillna(0)
    final_data.columns = ['GEOID'] + ['label{}'.format(i) for i in range(cnum)]+ ['CensusPop']
    final_data.to_csv('./AirbnbData/Chicago/finch-census-guided-targetindex{}-c.csv'.format(str(target_index)[5:]), index=False)


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from collections import defaultdict
from functools import cmp_to_key
from scipy.optimize import nnls


class Graph:
    def __init__(self,vertices):
        self.V= vertices
        self.graph = defaultdict(list)

    def addEdge(self,u,v):
        self.graph[u].append(v)

    def printPathsFunc(self, u, d, visited, path, current_path_list):
        visited[u]= True
        path.append(u)

        if u == d:
            path_copy = path[:]
            current_path_list.append(path_copy)
        else:
            for i in self.graph[u]:
                if visited[i]==False:
                    self.printPathsFunc(i, d, visited, path, current_path_list)

        path.pop()
        visited[u]= False
        return current_path_list


    def printPaths(self, s, d):
        total_results = []
        for start in s:
            for dest in d:
                path = []
                visited =[False]*(self.V)
                current_path_list = []
                current_path_results = self.printPathsFunc(start, dest, visited, path, current_path_list)
                if len(current_path_results) != 0:
                    total_results.extend(current_path_results)
        return total_results


def graph_process(config_path):
    cluster_unify = []
    partial_order = []
    start_candidates = []
    end_candidates = []

    f = open(config_path, 'r')
    while True:
        line = f.readline()
        if '=' in line:
            unify = list(map(int, line.split('=')))
            cluster_unify.append(unify)
        elif '<' in line:
            order = list(map(int, line.split('<')))
            partial_order.append(order)
            start_candidates.append(order[0])
            end_candidates.append(order[1])

        if not line: break
    f.close()

    start = []
    end = []
    for element in start_candidates:
        if element in end_candidates:
            continue
        start.append(element)

    for element in end_candidates:
        if element in start_candidates:
            continue
        end.append(element)

    start = list(set(start))
    end = list(set(end))
    return start, end, partial_order, cluster_unify



def generate_graph(partial_order_list, vertex_num):
    cluster_graph = Graph(vertex_num)
    for pair in partial_order_list:
        cluster_graph.addEdge(pair[0], pair[1])
    return cluster_graph


def save_graph_config(ordered_list, name):
    f = open("./AirbnbData/Chicago/{}".format(name), 'w+')

    for i in range(len(ordered_list) - 1):
        f.write('{}<{}\n'.format(ordered_list[i+1][0], ordered_list[i][0]))

    for orders in ordered_list:
        if len(orders) >= 2:
            f.write(str(orders[0]))
            for element in orders[1:]:
                f.write('={}'.format(element))
            f.write('\n')
    f.close()



def graph_inference_census(df, cluster_num, file_path):
    def numeric_compare(x, y):
        pop_list1 = result_list[x]
        pop_list2 = result_list[y]
        tTestResult = stats.ttest_ind(pop_list1, pop_list2)
        if (tTestResult.pvalue < 0.05) and (np.mean(pop_list1) < np.mean(pop_list2)):
            return 1
        elif (tTestResult.pvalue < 0.05) and (np.mean(pop_list1) >= np.mean(pop_list2)):
            return -1
        else:
            return 0

    result_list = []

    for i in range(cluster_num):
        result_list.append([])

    for _ in range(100):
        msk = np.random.rand(len(df)) < 0.5
        df_train = df[msk][(['label{}'.format(i) for i in range(cluster_num)]+['CensusPop'])]
        df_test = df[~msk][(['label{}'.format(i) for i in range(cluster_num)]+['CensusPop'])]

        # selected_hist_train = hist.loc[df_train['Directory'] - 1]
        # selected_hist_test = hist.loc[df_test['Directory'] - 1]

        train_X = df_train[['label{}'.format(i) for i in range(cluster_num)]].values
        train_y = df_train["CensusPop"].values
        test_X = df_train[['label{}'.format(i) for i in range(cluster_num)]].values
        test_y = df_test["CensusPop"].values
        result = nnls(train_X, train_y, 100)[0]
        for i in range(len(result)):
            result_list[i].append(result[i])

    sorted_list = sorted(range(cluster_num ), key=cmp_to_key(numeric_compare))
    ordered_list = []
    ordered_list.append([sorted_list[0]])
    curr = 0
    for i in range(len(sorted_list) - 1):
        if numeric_compare(sorted_list[i], sorted_list[i+1]) == 0:
            ordered_list[curr].append(sorted_list[i+1])
        else:
            curr += 1
            ordered_list.append([sorted_list[i+1]])

    # ordered_list.append([cluster_num])
    save_graph_config(ordered_list, file_path)
    return './graph_data/{}'.format(file_path)

for target_index in ['index002','index004','index006','index008','index01']:
        print(target_index)
        df = pd.read_csv('./AirbnbData/Chicago/finch-census-guided-targetindex{}-c.csv'.format(str(target_index)[5:]))
        graph_inference_census(df, cnum, "finch-graph-config-target{}-c.txt".format(target_index))


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from collections import defaultdict
from functools import cmp_to_key
from scipy.optimize import nnls


class Graph:
    def __init__(self,vertices):
        self.V= vertices
        self.graph = defaultdict(list)

    def addEdge(self,u,v):
        self.graph[u].append(v)

    def printPathsFunc(self, u, d, visited, path, current_path_list):
        visited[u]= True
        path.append(u)

        if u == d:
            path_copy = path[:]
            current_path_list.append(path_copy)
        else:
            for i in self.graph[u]:
                if visited[i]==False:
                    self.printPathsFunc(i, d, visited, path, current_path_list)

        path.pop()
        visited[u]= False
        return current_path_list


    def printPaths(self, s, d):
        total_results = []
        for start in s:
            for dest in d:
                path = []
                visited =[False]*(self.V)
                current_path_list = []
                current_path_results = self.printPathsFunc(start, dest, visited, path, current_path_list)
                if len(current_path_results) != 0:
                    total_results.extend(current_path_results)
        return total_results


def graph_process(config_path):
    cluster_unify = []
    partial_order = []
    start_candidates = []
    end_candidates = []

    f = open(config_path, 'r')
    while True:
        line = f.readline()
        if '=' in line:
            unify = list(map(int, line.split('=')))
            cluster_unify.append(unify)
        elif '<' in line:
            order = list(map(int, line.split('<')))
            partial_order.append(order)
            start_candidates.append(order[0])
            end_candidates.append(order[1])

        if not line: break
    f.close()

    start = []
    end = []
    for element in start_candidates:
        if element in end_candidates:
            continue
        start.append(element)

    for element in end_candidates:
        if element in start_candidates:
            continue
        end.append(element)

    start = list(set(start))
    end = list(set(end))
    return start, end, partial_order, cluster_unify



def generate_graph(partial_order_list, vertex_num):
    cluster_graph = Graph(vertex_num)
    for pair in partial_order_list:
        cluster_graph.addEdge(pair[0], pair[1])
    return cluster_graph


def save_graph_config(ordered_list, name):
    f = open("./AirbnbData/Chicago/{}".format(name), 'w+')

    for i in range(len(ordered_list) - 1):
        f.write('{}<{}\n'.format(ordered_list[i+1][0], ordered_list[i][0]))

    for orders in ordered_list:
        if len(orders) >= 2:
            f.write(str(orders[0]))
            for element in orders[1:]:
                f.write('={}'.format(element))
            f.write('\n')
    f.close()



def graph_inference_census(df, cluster_num, file_path):
    def numeric_compare(x, y):
        pop_list1 = result_list[x]
        pop_list2 = result_list[y]
        tTestResult = stats.ttest_ind(pop_list1, pop_list2)
        if (tTestResult.pvalue < 0.05) and (np.mean(pop_list1) < np.mean(pop_list2)):
            return 1
        elif (tTestResult.pvalue < 0.05) and (np.mean(pop_list1) >= np.mean(pop_list2)):
            return -1
        else:
            return 0

    result_list = []

    for i in range(cluster_num):
        result_list.append([])

    for _ in range(100):
        msk = np.random.rand(len(df)) < 0.5
        df_train = df[msk][(['label{}'.format(i) for i in range(cluster_num)]+['CensusPop'])]
        df_test = df[~msk][(['label{}'.format(i) for i in range(cluster_num)]+['CensusPop'])]

        # selected_hist_train = hist.loc[df_train['Directory'] - 1]
        # selected_hist_test = hist.loc[df_test['Directory'] - 1]

        train_X = df_train[['label{}'.format(i) for i in range(cluster_num)]].values
        train_y = df_train["CensusPop"].values
        test_X = df_train[['label{}'.format(i) for i in range(cluster_num)]].values
        test_y = df_test["CensusPop"].values
        result = nnls(train_X, train_y, 100)[0]
        for i in range(len(result)):
            result_list[i].append(result[i])

    sorted_list = sorted(range(cluster_num ), key=cmp_to_key(numeric_compare))
    ordered_list = []
    ordered_list.append([sorted_list[0]])
    curr = 0
    for i in range(len(sorted_list) - 1):
        if numeric_compare(sorted_list[i], sorted_list[i+1]) == 0:
            ordered_list[curr].append(sorted_list[i+1])
        else:
            curr += 1
            ordered_list.append([sorted_list[i+1]])

    # ordered_list.append([cluster_num])
    save_graph_config(ordered_list, file_path)
    return './AirbnbData/Chicago/{}'.format(file_path)


In [ ]:
import numpy as np
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
import pandas as pd
import warnings
from scipy.stats import spearmanr, pearsonr
import pickle
from datetime import datetime
from numpy import nanmin, nanmax, nanmean, nanvar, nanmedian
from sklearn.impute import SimpleImputer
from scipy.stats import spearmanr, pearsonr
from sklearn import preprocessing


class TestDataset(Dataset):
    def __init__(self, target_index, cnum):
        test_data = pd.read_csv('./NYC_Experiments/data/val-gridsize-{}.csv'.format(str(target_index)[5:]))
        train_data = pd.read_csv('./AirbnbData/Chicago/train-gridsize-{}.csv'.format(str(target_index)[5:]))
        test_data.drop(['label', target_index], axis=1, inplace=True)
        train_data.drop(['label', target_index], axis=1, inplace=True)

        X_train =  train_data.values
        imputer = SimpleImputer(strategy='mean')
        imputer_model = imputer.fit(X_train)
        X_train = imputer_model.transform(X_train)

        X_test = imputer_model.transform(test_data.values)

        # min-max scaling
        min_max_scaler_model = (preprocessing.MinMaxScaler()).fit(X_train)
        X_train = min_max_scaler_model.transform(X_train)
        X_test = min_max_scaler_model.transform(X_test)

        self.file_list = X_test

    def __len__(self):
        return (self.file_list).shape[0]

    def __getitem__(self, idx):
        return self.file_list[idx], idx


from sklearn.utils import resample

class ClusterDataset(Dataset):
    def __init__(self, cluster_list, target_index, cnum, batch_sz):
        self.data_list = None
        train_data_class = pd.read_csv('./AirbnbData/Chicago/finch-train-gridsize-{}-c.csv'.format(str(target_index)[5:]))[['cluster_labels']]
        train_data_class['cluster_labels'] = train_data_class['cluster_labels'].map(lambda x: int(x))
        train_data = pd.read_csv('./AirbnbData/Chicago/train-gridsize-{}.csv'.format(str(target_index)[5:]))
        train_data.drop(['label', target_index], axis=1, inplace=True)

        X_train =  train_data.values
        imputer = SimpleImputer(strategy='mean')
        imputer_model = imputer.fit(X_train)
        X_train = imputer_model.transform(X_train)
        # min-max scaling
        min_max_scaler_model = (preprocessing.MinMaxScaler()).fit(X_train)
        X_train = min_max_scaler_model.transform(X_train)
        train_data_v2 = pd.DataFrame(X_train, columns = train_data.columns)

        for cluster_num in cluster_list:
            tmp =train_data_v2[train_data_class['cluster_labels']==int(cluster_num)]
            if self.data_list is None:
                self.data_list = tmp.values
            else:
                self.data_list = np.concatenate([self.data_list, tmp.values])

        if (self.data_list.shape[0]<batch_sz) and (self.data_list.shape[0] > 0):
              self.data_list = (resample(pd.DataFrame(self.data_list), n_samples=batch_sz,replace=True)).values

    def __len__(self):
        return (self.data_list).shape[0]

    def __getitem__(self, idx):
        return self.data_list[idx]


In [ ]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import pandas as pd
from torch.utils.data import DataLoader
import argparse
from itertools import permutations

class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(181, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 16),
      nn.ReLU(),
      nn.Linear(16, 8),
      nn.ReLU(),
      nn.Linear(8, 1),
      nn.ReLU()
    )
    self.layers.cuda()
    for m in self.layers.modules():
      if isinstance(m, nn.Linear):
          m.weight.data = m.weight.data.double()
          m.bias.data = m.bias.data.double()


  def forward(self, x):
    return self.layers(x)


def make_data_loader(cluster_list, batch_sz, target_index, cluster_num):
    cluster_dataset = ClusterDataset(cluster_list, target_index, cluster_num, batch_sz)
    if cluster_dataset.__len__() == batch_sz:
        cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=batch_sz, shuffle=True, num_workers=4, drop_last=False)
    else:
        cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=batch_sz, shuffle=True, num_workers=4, drop_last=True)
    return cluster_loader


def generate_loader_dict(total_list, unified_cluster_list, batch_sz, target_index, cluster_num):
    loader_dict = {}
    for cluster_id in total_list:
        cluster_loader = make_data_loader([cluster_id], batch_sz, target_index, cluster_num)
        loader_dict[cluster_id] = cluster_loader

    for cluster_tuple in unified_cluster_list:
        cluster_loader = make_data_loader(cluster_tuple, batch_sz, target_index, cluster_num)
        for cluster_id in cluster_tuple:
            loader_dict[cluster_id] = cluster_loader
    return loader_dict


def train(epoch, model, optimizer, loader_list, cluster_path_list, device, lr, batchsz, seed, lamb, alpha):
    model.train()
    train_loss = AverageMeter()
    reg_loss = AverageMeter()

    # For each cluster route
    path_idx = 0
    avg_loss = 0
    count = 0
    for cluster_path in cluster_path_list:
        path_idx += 1
        dataloaders = []
        for cluster_id in cluster_path:
            dataloaders.append(loader_list[cluster_id])

        for batch_idx, data in enumerate(zip(*dataloaders)):
            cluster_num = len(data)
            data_zip = torch.cat(data, 0).to(device)

            # Generating Score
            scores = model(data_zip).squeeze()
            scores = torch.clamp(scores, min=0, max=1)
            score_list = torch.split(scores, batchsz, dim = 0)

            # Standard deviation as a loss
            loss_var = torch.zeros(1).to(device)
            for score in score_list:
                loss_var += score.var()
            loss_var /= len(score_list)

            # Differentiable Ranking with sigmoid function
            rank_matrix = torch.zeros((batchsz, cluster_num, cluster_num)).to(device)
            for itertuple in list(permutations(range(cluster_num), 2)):
                score1 = score_list[itertuple[0]]
                score2 = score_list[itertuple[1]]
                diff = lamb * (score2 - score1)
                results = torch.sigmoid(diff)
                rank_matrix[:, itertuple[0], itertuple[1]] = results
                rank_matrix[:, itertuple[1], itertuple[0]] = 1 - results

            rank_predicts = rank_matrix.sum(1)
            temp = torch.Tensor(range(cluster_num))
            target_rank = temp.unsqueeze(0).repeat(batchsz, 1).to(device)

            # Equivalent to spearman rank correlation loss
            loss_train = ((rank_predicts - target_rank)**2).mean()
            loss = loss_train + loss_var * alpha
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.update(loss_train.item(), batchsz)
            reg_loss.update(loss_var.item(), batchsz)
            avg_loss += loss.item()
            count += 1


            # Print status
            if batch_idx % 10 == 0:
                print('Epoch: [{epoch}][{path_idx}][{elps_iters}] '
                      'Train loss: {train_loss.val:.4f} ({train_loss.avg:.4f}) '
                      'Reg loss: {reg_loss.val:.4f} ({reg_loss.avg:.4f})'.format(
                          epoch=epoch, path_idx=path_idx, elps_iters=batch_idx, train_loss=train_loss, reg_loss=reg_loss))

    return avg_loss / count


def main(lr, batchsz, epochs, seed, lamb, alpha, cluster_num, target_index, modelname):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    graph_config = "./AirbnbData/Chicago/finch-graph-config-target{}-c.txt".format(target_index)
    start, end, partial_order, cluster_unify = graph_process(graph_config)

    loader_list = generate_loader_dict(range(cluster_num), cluster_unify, batchsz, target_index, cluster_num)
    cluster_graph = generate_graph(partial_order, cluster_num)
    cluster_path_list = cluster_graph.printPaths(start, end)
    print("Cluster_path: ", cluster_path_list)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = MLP()
    model.to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    valid_data = pd.read_csv('./NYC_Experiments/data/val-gridsize-{}.csv'.format(str(target_index)[5:]))[['label']]

    best_spearman_corr = -2
    patience = 0
    for epoch in range(epochs):
            loss = train(epoch, model, optimizer, loader_list, cluster_path_list, device, lr, batchsz, seed, lamb, alpha)

            model.eval()

            test_dataset = TestDataset(target_index, cluster_num)
            test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchsz, shuffle=False, num_workers=4)

            prediction_list = []
            with torch.no_grad():
                for batch_idx, (data, name) in enumerate(test_loader):
                    data = data.to(device)
                    scores = model(data).squeeze()
                    count = 0
                    for each_name in name:
                        prediction_list.append(scores[count].cpu().data.numpy())
                        count += 1

            spearman_corr = spearmanr(np.array(prediction_list), valid_data.values).statistic
            print(spearman_corr)

            if best_spearman_corr < spearman_corr:
                    patience = 0
                    print("state saving...")
                    state = {
                        'model': model.state_dict(),
                        'loss': loss
                    }
                    if not os.path.isdir('./AirbnbData/Chicago/checkpoint'):
                        os.mkdir('./AirbnbData/Chicago/checkpoint')
                    torch.save(state, './AirbnbData/Chicago/checkpoint/{}'.format(modelname))
                    best_spearman_corr = spearman_corr
            else:
                    patience += 1
                    if patience > 10:
                      break

for target_index in ['index002','index004','index006','index008','index01']:
      dummy = pd.read_csv('./AirbnbData/Chicago/finch-train-gridsize-{}-c.csv'.format(str(target_index)[5:]))[['cluster_labels']]
      cluster_num = len(list(set(list(train_data_class['cluster_labels'].values))))
      print(target_index, cluster_num)
      main(lr = 0.0002, batchsz=32, epochs=1000, seed=1567010775, lamb=30, alpha=4, cluster_num=cluster_num, target_index=target_index, modelname = 'finch-checkpoint_target_{}_c.pth'.format(target_index))


In [ ]:
import numpy as np
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
import pandas as pd
import warnings
from scipy.stats import spearmanr, pearsonr
import pickle
from datetime import datetime
from numpy import nanmin, nanmax, nanmean, nanvar, nanmedian
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from torch.utils.data import DataLoader
import argparse
from itertools import permutations


class TestDataset(Dataset):
    def __init__(self, target_index, cnum):
        test_data = pd.read_csv('./NYC_Experiments/data/test-gridsize-{}.csv'.format(str(target_index)[5:]))
        train_data = pd.read_csv('./AirbnbData/Chicago Airbnb 07:2023/train-gridsize-{}.csv'.format(str(target_index)[5:]))
        test_data.drop(['label', target_index], axis=1, inplace=True)
        train_data.drop(['label', target_index], axis=1, inplace=True)

        X_train =  train_data.values
        imputer = SimpleImputer(strategy='mean')
        imputer_model = imputer.fit(X_train)
        X_train = imputer_model.transform(X_train)

        X_test = imputer_model.transform(test_data.values)

        # min-max scaling
        min_max_scaler_model = (preprocessing.MinMaxScaler()).fit(X_train)
        X_train = min_max_scaler_model.transform(X_train)
        X_test = min_max_scaler_model.transform(X_test)

        self.file_list = X_test

    def __len__(self):
        return (self.file_list).shape[0]

    def __getitem__(self, idx):
        return self.file_list[idx], idx



class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(181, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 16),
      nn.ReLU(),
      nn.Linear(16, 8),
      nn.ReLU(),
      nn.Linear(8, 1),
      nn.ReLU()
    )
    self.layers.cuda()
    for m in self.layers.modules():
      if isinstance(m, nn.Linear):
          m.weight.data = m.weight.data.double()
          m.bias.data = m.bias.data.double()


  def forward(self, x):
    return self.layers(x)


def make_data_loader(cluster_list, batch_sz, target_index, cluster_num):
    cluster_dataset = ClusterDataset(cluster_list, target_index, cluster_num, batch_sz)
    if cluster_dataset.__len__() == batch_sz:
        cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=batch_sz, shuffle=True, num_workers=4, drop_last=False)
    else:
        cluster_loader = torch.utils.data.DataLoader(cluster_dataset, batch_size=batch_sz, shuffle=True, num_workers=4, drop_last=True)
    return cluster_loader


def generate_loader_dict(total_list, unified_cluster_list, batch_sz, target_index, cluster_num):
    loader_dict = {}
    for cluster_id in total_list:
        cluster_loader = make_data_loader([cluster_id], batch_sz, target_index, cluster_num)
        loader_dict[cluster_id] = cluster_loader

    for cluster_tuple in unified_cluster_list:
        cluster_loader = make_data_loader(cluster_tuple, batch_sz, target_index, cluster_num)
        for cluster_id in cluster_tuple:
            loader_dict[cluster_id] = cluster_loader
    return loader_dict



def main(lr, batchsz, epochs, seed, lamb, alpha, cluster_num, target_index, modelname):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = MLP()
    model.load_state_dict(torch.load('./AirbnbData/Chicago/checkpoint/{}'.format(modelname))['model'])
    model.to(device)

    valid_data = pd.read_csv('./NYC_Experiments/data/test-gridsize-{}.csv'.format(str(target_index)[5:]))[['label']]
    train_tmp = pd.read_csv('./NYC_Experiments/data/train-gridsize-{}.csv'.format(str(target_index)[5:]))[['label']]
    train_y = train_tmp.values
    valid_data = np.squeeze((np.array(valid_data.values) - np.min(train_y))/(np.max(train_y)-np.min(train_y)))
    model.eval()

    test_dataset = TestDataset(target_index, cluster_num)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchsz, shuffle=False, num_workers=4)

    prediction_list = []
    with torch.no_grad():
        for batch_idx, (data, name) in enumerate(test_loader):
              data = data.to(device)
              scores = model(data).squeeze()
              count = 0
              for each_name in name:
                  if len(name) > 1:
                    prediction_list.append(scores[count].cpu().data.numpy())
                  else:
                    prediction_list.append(scores.cpu().data.numpy())
                  count+=1

    x, y = np.array(prediction_list).astype(float), valid_data.astype(float)

    print(spearmanr(np.array(prediction_list), valid_data).statistic, pearsonr(x, y).statistic)


for target_index in ['index002','index004','index006','index008','index01']:
      dummy = pd.read_csv('./AirbnbData/Chicago/finch-train-gridsize-{}-c.csv'.format(str(target_index)[5:]))[['cluster_labels']]
      cluster_num = len(list(set(list(train_data_class['cluster_labels'].values))))
      print(target_index, cluster_num)
      main(lr = 0.0002, batchsz=32, epochs=1000, seed=1567010775, lamb=30, alpha=4, cluster_num=cluster_num, target_index=target_index, modelname = 'finch-checkpoint_target_{}_c.pth'.format(target_index))